In [ ]:
# Ipython 
import matplotlib.pyplot as plt
from sympy.physics.vector import init_vprinting
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
init_vprinting(use_latex='mathjax', pretty_print=False)
# Sympy
import numpy as np
import sympy as sp
from sympy import Matrix, symbols, simplify, Function, expand_trig, Symbol, diff
from sympy import cos, sin, transpose, pi
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point, inertia
# Yams sympy
from welib.yams.yams_sympy       import YAMSRigidBody, YAMSInertialBody, YAMSFlexibleBody
from welib.yams.yams_sympy_model import YAMSModel

In [ ]:
m, k, c = symbols('m, k, c')
gravity = symbols('g')
time = dynamicsymbols._t
x, y, z = dynamicsymbols('x, y, z')
xd, yd, zd = dynamicsymbols('xd, yd, zd')
# --- Bodies
ref = YAMSInertialBody('E') 
body = YAMSRigidBody('B') # Create a body named "B"
bodies=[body] 
# --- Points
P_O = body.origin

# --- Connections
ref.connectTo(body, type='Free', rel_pos=(0, 0, z))

# --- DOFs and kinetic equations
coordinates = [z]
speeds      = [zd]
kdeqsSubs = []
for q,qd in zip(coordinates, speeds):
    kdeqsSubs += [(qd, diff(q, time))]
#print('coordinates:', coordinates)
#print('kdeqsSubs  :', kdeqsSubs)

# --- Loads
body_loads = []
for bdy in bodies: # gravity
    body_loads  += [(bdy, (bdy.masscenter, -bdy.mass * gravity * ref.frame.z))]
fr =0
fr += -k * z            * ref.frame.z # Restoring force
fr += -c * z.diff(time) * ref.frame.z # Damping force
body_loads += [(body, (P_O,  fr))]
    
# --- Model and equations of motions
model = YAMSModel(name='MSD', ref=ref, bodies=bodies, body_loads=body_loads, 
                  coordinates=coordinates, speeds=speeds, kdeqsSubs=kdeqsSubs, g_vect=-gravity*ref.frame.z)
#model.addForce(body, P_O, fr)
#model.addMoment(body, body.frame, Mr)

extraSubs=[]

EOM=model.to_EOM()
print('Equations of motions')
EOM.EOM
EOM.mass_forcing_form()
print('Mass matrix:')
EOM.M
print('Forcing term:')
EOM.F
#M0, C0, K0, B0 = EOM = EOM.linearize(op_point=None, noAcc=True, noVel=False, extraSubs=None)
#M0, C0, K0, B0

In [ ]:
# Export to python code
print(EOM.toPython())
#EOM.savePython('_TuneMassDamper.py')